In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#from sklearn.datasets import load_boston
# 데이터 불러오기
#boston = load_boston()
#print(boston)
# 입력 데이터의 데이터 프레임
#X = pd.DataFrame(boston.data, columns=boston.feature_names)
#print(X)
# CHAS 열 삭제
#X = X.drop(['CHAS'],axis=1)
#print(X.head())
# 목표변수의 데이터 프레임
#y = pd.DataFrame(boston.target)
#print(y)

In [ ]:
# 데이터 불러오기
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [ ]:
# 입력 데이터의 데이터 프레임
X = pd.DataFrame(data)
print(X)

In [ ]:
# 3 열 삭제
X = X.drop(X.columns[3],axis=1)
print(X.head())

In [ ]:
# 목표변수의 데이터 프레임
y = pd.DataFrame(target)
print(y)

In [ ]:
# 학습용과 데이터 분리(7:3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# 표준화 스케일러 생성
scalerX = MinMaxScaler()
# 표준화 스케일러를 학습용 데이터에 맞춤
scalerX.fit(X_train)
# 표준화 스케일러로 학습용 데이어를 변환
X_train_norm = scalerX.transform(X_train)
# 표준화 스케일러로 테스트용 데이어를 변환
X_test_norm = scalerX.transform(X_test)

In [ ]:
# 표준화 스케일러 생성
scalerY = MinMaxScaler()
# 표준화 스케일러를 학습용 데이터에 맞춤
scalerY.fit(y_train)
# 표준화 스케일러로 학습용 데이어를 변환
y_train_norm = scalerY.transform(y_train)
# 표준화 스케일러로 테스트용 데이어를 변환
y_test_norm = scalerY.transform(y_test)

In [ ]:
# 모형 구조: 은닉층 노드의 드롭 아웃
model = Sequential()
# 제1은닉층의 60개 노드와 12개 입력 노드
model.add(Dense(60, activation='relu', input_shape=(12,)))
# 제1은닉층과 제2은닉층 사이의 드롭 아웃(50%)
model.add(Dropout(0.5))
# 제2은닉층의 60개 노드
model.add(Dense(60, activation='relu'))
# 제3은닉층의 30개 노드
model.add(Dense(30, activation='relu'))
# 제3은닉층과 출력층 사이의 드롭 아웃(20%)
model.add(Dropout(0.2))
# 1개 출력 노드
model.add(Dense(1))

In [ ]:
# 모형 구조 출력
model.summary()

In [ ]:
# 모형 구성
model.compile(optimizer='adam',
             loss='mse',
             metrics=['mae'])

In [ ]:
# 학습
results = model.fit(X_train_norm, y_train_norm,
                    validation_data=(X_test_norm, y_test_norm),
                    epochs=200, batch_size=32)

In [ ]:
# 학습모형에 저장된 키(변수)
print(results.history.keys())

In [ ]:
# 학습 수에 따른 loss 변화
plt.figure(figsize=(10, 5))
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'validate'], loc='upper right')
plt.show()

In [ ]:
# 학습 수에 따른 정확도(mae) 변화
plt.figure(figsize=(10, 5))
plt.plot(results.history['mae'])
plt.plot(results.history['val_mae'])
plt.title('accuracy(MAE)')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train', 'validate'], loc='upper right')
plt.show()

In [ ]:
# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test_norm).flatten()
# 예측 깂의 역변환
y_pred_inverse = scalerY.inverse_transform(y_pred.reshape(-1,1))
print(y_pred_inverse[0:10])

In [ ]:
# 오차 측정(MAE)
print('MAE: %.2f' % mean_absolute_error(y_test, y_pred_inverse))

In [ ]:
# 실제 값 대비 예측 값의 산포도
plt.figure(figsize=(7, 7))
plt.scatter(y_test_norm, y_pred, c='r')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.axis('equal')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.plot([0, 1], [0, 1])
plt.show()